# extracting gps from videos for 5 meters

In [1]:
import cv2
import pytesseract
import re
import os

# === CONFIG ===
VIDEO_PATH = 'D:\\gps_vid\\a'
FRAME_INTERVAL = 30  # Process every 30th frame (adjust based on fps and performance)
OUTPUT_CSV = 'extracted_gps_from_ocr.csv'

# === TESSERACT CONFIG ===
# Set your Tesseract-OCR path here if needed (only for Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# === OCR SETTINGS ===
custom_config = r'--oem 3 --psm 6'

# === FUNCTION TO PARSE GPS FROM TEXT ===
def parse_gps(text):
    """
    Attempt to extract latitude and longitude from OCR text.
    Supports formats like:
        37.7749 N, 122.4194 W
        Lat: 37.7749, Lon: -122.4194
    """
    # Remove newlines and convert to one-liner
    text = text.replace('\n', ' ')
    
    # Try multiple regex patterns
    patterns = [
        r'(-?\d+\.\d+)[^\d\-]+(-?\d+\.\d+)',   # e.g., 37.7749, -122.4194
        r'Lat[: ]+(-?\d+\.\d+)[ ,]+Lon[: ]+(-?\d+\.\d+)',  # Lat: 37.7749, Lon: -122.4194
    ]

    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            lat, lon = float(match.group(1)), float(match.group(2))
            return lat, lon
    return None

# === PROCESS VIDEO ===
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Video FPS: {fps}, Total Frames: {frame_count}")

frame_idx = 0
extracted_data = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_idx % FRAME_INTERVAL == 0:
        # Convert frame to grayscale for better OCR
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Run OCR on the frame
        text = pytesseract.image_to_string(gray, config=custom_config)
        gps = parse_gps(text)

        if gps:
            timestamp = frame_idx / fps
            print(f"Frame {frame_idx}: Time {timestamp:.2f}s, GPS: {gps}")
            extracted_data.append({
                'frame': frame_idx,
                'timestamp_sec': round(timestamp, 2),
                'latitude': gps[0],
                'longitude': gps[1]
            })

    frame_idx += 1

cap.release()

# === SAVE TO CSV ===
import pandas as pd
df = pd.DataFrame(extracted_data)
df.to_csv(OUTPUT_CSV, index=False)
print(f"Saved extracted GPS data to: {OUTPUT_CSV}")


# extracting 5m frames

In [ ]:
import cv2
import pandas as pd
import os
from math import radians, sin, cos, sqrt, atan2

# === CONFIG ===
VIDEO_PATH = 'E:\\JupyterNotebooks\\Road_problem\\gps_vid\\b.mp4'
GPS_CSV_PATH = 'extracted_gps_from_ocr.csv'
OUTPUT_FOLDER = 'E:\\JupyterNotebooks\\Road_problem\\frames_5m'
DISTANCE_THRESHOLD_METERS = 5  # Save frame every 5 meters

os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# === HAVERSINE FORMULA ===
def haversine(coord1, coord2):
    R = 6371000  # Radius of Earth in meters
    lat1, lon1 = radians(coord1[0]), radians(coord1[1])
    lat2, lon2 = radians(coord2[0]), radians(coord2[1])
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# === LOAD GPS DATA ===
gps_data = pd.read_csv(GPS_CSV_PATH)

# === OPEN VIDEO ===
cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)

prev_coord = None
saved_frames = 0

for idx, row in gps_data.iterrows():
    curr_time = row['timestamp_sec']
    curr_coord = (row['latitude'], row['longitude'])

    if prev_coord is None:
        prev_coord = curr_coord
        continue

    distance = haversine(prev_coord, curr_coord)

    if distance >= DISTANCE_THRESHOLD_METERS:
        # Seek to frame corresponding to this timestamp
        frame_number = int(curr_time * fps)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        ret, frame = cap.read()

        if ret:
            filename = f"frame_{saved_frames:04d}.jpg"
            filepath = os.path.join(OUTPUT_FOLDER, filename)
            cv2.imwrite(filepath, frame)
            print(f"Saved: {filename}, Time: {curr_time:.2f}s, Distance: {distance:.2f}m")
            saved_frames += 1
            prev_coord = curr_coord

cap.release()
print(f"Total frames saved: {saved_frames}")